In [ ]:
using Revise, RecursiveArrayTools, ReachabilityAnalysis

In [ ]:
sol1 = solve(@ivp(x' = -x, x(0) ∈ 0.5 .. 0.6), tspan=(0.0, 5.0));
sol2 = solve(@ivp(x' = -x, x(0) ∈ 0.6 .. 0.7), tspan=(0.0, 5.0));

In [ ]:
fp = HybridFlowpipe([sol1, sol2]);

In [ ]:
dim(fp[1, 1])

In [ ]:
v = VectorOfArray([sol1, sol2]);

In [ ]:
typeof(v)

In [ ]:
#VectorOfArray{RT, N, Flowpipe{N, RT}}([sol1, sol2])

In [ ]:
N = Float64
RT = ReachSet{Float64,Interval{Float64,IntervalArithmetic.Interval{Float64}}}

voa = VectorOfArray{RT, 2, Vector{Flowpipe{N, RT}}}([sol1, sol2]);

In [ ]:
d = Dict("a"=>1.0, "b"=>2.0)

In [ ]:
d["c"] = 34.0

In [ ]:
d

In [ ]:
Base.size(fp::Flowpipe) = (length(fp),)

In [ ]:
t = (1, 2, 3.0)
typeof(t)

In [ ]:
size([])

In [ ]:
?NTuple

In [ ]:
a1 = [1, 2, 3]
a2 = [4, 5]

In [ ]:
@which VectorOfArray([a1, a2])

In [ ]:
size(v)

In [ ]:
@which VectorOfArray([sol1, sol2])

In [ ]:
v[1, 1]

In [ ]:
typeof(v)

In [ ]:
typeof(sol1)

In [ ]:
typeof(sol2)

In [ ]:
a1 = [rand(2, 2) for _ in 1:3]
a2 = [rand(Float32, 3) for _ in 1:3]
x = vcat(a1, a2)

In [ ]:
typeof(size(a1[1])) == NTuple{2, Int}

In [ ]:
typeof((3, 3, 4)) == NTuple{3, Int}

In [ ]:
v = VectorOfArray(x)

In [ ]:
typeof(v)

In [ ]:
x[1][1:2]

In [ ]:
v[1:2, 1]

In [ ]:
x[1, 1]

In [ ]:
typeof(v)

In [ ]:
v[1]

In [ ]:
v[2]

In [ ]:
v[3]

In [ ]:
v[4]

In [ ]:
v[1, 1]

In [ ]:
v[2, 1]

In [ ]:
for vi in v
    println(vi)
end

In [ ]:
for vi in v
    for X in vi
        println(X)
    end
end

## Evaluation-like behavior

### Initial version for points

In [ ]:
# initial version
function (fp::HybridFlowpipe)(t::Number)
    Fk = array(fp)
    @inbounds for (i, F) in enumerate(Fk)
        for (j, X) in enumerate(F)
            if t ∈ tspan(X) # exit on the first occurrence
                if j < length(F) && t ∈ tspan(F[i+1])
                    return view(Fk, j:j+1, i)
                else
                    return X
                end
            end
        end
    end
    throw(ArgumentError("time $t does not belong to the time span, " *
                        "$(tspan(fp)), of the given flowpipe"))
end

### Faster version for points

In [ ]:
# faster version: first consider the time span of each
# outer flowpipe
function _find(fp::HybridFlowpipe, t::Number)
    Fk = array(fp)
    i = 1
    while t ∉ tspan(Fk[i])
        i += 1
    end
    i > length(Fk) && @goto error_msg
    F = Fk[i]
    @inbounds for (j, X) in enumerate(F)
        if t ∈ tspan(X) # exit on the first occurrence
            if j < length(F) && t ∈ tspan(F[i+1])
                return view(Fk, j:j+1, i)
            else
                return X
            end
        end
    end
    @label error_msg
    throw(ArgumentError("time $t does not belong to the time span, " *
                        "$(tspan(fp)), of the given flowpipe"))
end

---

In [1]:
using Revise, ReachabilityAnalysis, BenchmarkTools
const RA = ReachabilityAnalysis
using Plots
include("/home/mforets/.julia/dev/ReachabilityAnalysis/test/models/hybrid/embrake.jl")

┌ Info: Precompiling ReachabilityAnalysis [1e97bd63-91d1-579d-8e8d-501d2b57c93f]
└ @ Base loading.jl:1260


embrake_pv_2 (generic function with 1 method)

In [2]:
prob = embrake_no_pv(ζ=0.0, Tsample=1e-4);

In [3]:
@time sol = solve(prob, alg=GLGM06(δ=1e-8), max_jumps=20);

 11.436513 seconds (38.25 M allocations: 1.944 GiB, 4.89% gc time)


In [4]:
fp = flowpipe(sol);
typeof(fp)

HybridFlowpipe{Float64,ReachSet{Float64,Zonotope{Float64,Array{Float64,1},Array{Float64,2}}},Flowpipe{Float64,ReachSet{Float64,Zonotope{Float64,Array{Float64,1},Array{Float64,2}}}}}

In [5]:
similar(fp)

HybridFlowpipe{Float64,ReachSet{Float64,Zonotope{Float64,Array{Float64,1},Array{Float64,2}}},Flowpipe{Float64,ReachSet{Float64,Zonotope{Float64,Array{Float64,1},Array{Float64,2}}}}}(Flowpipe{Float64,ReachSet{Float64,Zonotope{Float64,Array{Float64,1},Array{Float64,2}}}}[], Dict{Symbol,Any}())

In [8]:
@which sol.F.Fk[1:10, 1]

getindex(A::AbstractArray, I...) in Base at abstractarray.jl:978

In [ ]:
similar(fp)

In [ ]:
s = solve(@ivp(x' = rand(2, 2) * x, x(0) ∈ rand(Hyperrectangle, dim=2)), T=5.0);

In [ ]:
p = project(s, (0, 1));

In [ ]:
s.F;

In [ ]:
overapproximate(s, Hyperrectangle)

In [ ]:
function find_dt(fp::HybridFlowpipe)(dt::TimeInterval)
    # here we assume that indices are one-based, ie. form 1 .. n
    firstidx = 0
    lastidx = 0
    α = inf(dt)
    β = sup(dt)
    Fk = array(fp)

    # search first intersecting flowpipe
    i = 1
    while α ∉ tspan(Fk[i])
        i += 1
    end
    i > length(Fk) && @goto error_msg
    firstfp = i

    # search last intersecting flowpipe
    i = 1
    while t ∉ tspan(Fk[i])
        i += 1
    end
    i > length(Fk) && @goto error_msg
    firstfp = i

    for (i, X) in enumerate(Xk)
        if α ∈ tspan(X)
            firstidx = i
        end
        if β ∈ tspan(X)
            lastidx = i
        end
    end
    if firstidx == 0 || lastidx == 0
        throw(ArgumentError("the time interval $dt is not contained in the time span, " *
                            "$(tspan(fp)), of the given flowpipe"))
    end
    return view(Xk, firstidx:lastidx)
end